In [14]:
# MODULE 6 CHALLENGE - PART 1
#  implement the feedback from the beta testers:
# - A weather description to the pop-up markers for customers so that they know what the weather is as they are traveling
# - A notation in the search criteria to indicate if it is raining or snowing for customers who are making travel decisions in real-time
# - A map that shows the directions for customers’ travel itinerary

# OBJECTIVES
# Use nested try-except blocks
# Use Pandas methods & attributes on a DF or Series
# Create a new DataFrame from a new API search with new weather parameters.
# Filter DataFrames based on input and nested decision statements, and logical expressions.
# Create pop-up markers on a Google map from a filtered DataFrame.
# Add a directions layer on a Google map between cities in the filtered DataFrame.

# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the requests library. 
# Run from terminal: - conda install -c conda-forge requests
import requests
#requests.__version__

# Import the API key.
from config import weather_api_key

# Import the datetime module from the datetime library.
from datetime import datetime

# Import linear regression from the SciPy stats module.
from scipy.stats import linregress

In [2]:
# Part 1 Instructions
# Get the Weather Description and Amount of Precipitation for Each City

# 1. Create a new Jupyter Notebook file and name it Weather_Database.ipynb. - THIS FILE!!
# 2. Generate a new set of 1,500 random latitudes and longitudes.
# 3. Get the nearest city using the citipy module.
# 4. Perform an API call with the OpenWeatherMap.
# - Retrieve the following information from the API call:
# -- Latitude and longitude
# -- Maximum temperature
# -- Percent humidity
# -- Percent cloudiness
# -- Wind speed
# -- Weather description (e.g., clouds, fog, light rain, clear sky)
# -- The amount of rainfall over the last hour (1 hr).
# -- The amount of snowfall over the last hour (1 hr)
# 5. Try-except: Use multiple try-except blocks to get the amount of rainfall and snowfall. In the except block handle the KeyError if there is no rainfall or snowfall and add 0 inches for the amount.
# 6. Add the data to a new DataFrame.
# 7. Save the new DataFrame as a CSV file to be used for Part 2.
# 8. Upload the CSV file as part of your submission as WeatherPy_database.csv.


In [3]:
# 2. Generate a new set of 1,500 random latitudes and longitudes.
# Create a set of random latitude and longitude combinations.
# Pack the latitudes (lats) and longitudes (lngs) as pairs by zipping them (lat_lngs) with the zip() function.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# 3. Get the nearest city using the citipy module.
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
# Use the coordinates in our lat_lngs tuple to find the nearest city using Python’s citipy module.
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
#print(cities)
#len(cities)

In [10]:
# 4. Perform an API call with the OpenWeatherMap.

# Starting URLs for Weather Map API Call.
city_base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
weather_base_url = "https://api.openweathermap.org/data/2.5/onecall?exclude=minutely,hourly,daily&appid="  + weather_api_key
#print(city_base_url)
#print(weather_base_url)

# - Retrieve the following information from the API call:
# -- Latitude and longitude
# -- Maximum temperature
# -- Percent humidity
# -- Percent cloudiness
# -- Wind speed
# -- Weather description (e.g., clouds, fog, light rain, clear sky)
# -- The amount of rainfall over the last hour (1 hr).
# -- The amount of snowfall over the last hour (1 hr)

# Create an empty list to hold the weather data.
city_weather_data = [] 
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
total_count = 0

# 5. Try-except: Use multiple try-except blocks to get the amount of rainfall and snowfall. In the except block handle the KeyError if there is no rainfall or snowfall and add 0 inches for the amount.
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    total_count += 1
    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = city_base_url + "&q=" + city.replace(" ","+")
    #print(city_url)

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Make the City API call, parse the JSON response (assuming you get the expected response) and retrieve data.
        city_weather = requests.get(city_url).json()
        #print(city_weather)

        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Weather description (e.g., clouds, fog, light rain, clear sky)
        # - Descrption is an ARRAY, may be more than one item, e.g. city_weather["weather"][0]["description"]
        # Reset a string to concatenate one or multiple 'description' strings.
        city_weather_desc = ''
        # Create a separator to distinguish between multiple descriptions.
        separator=''
        # Enumerate through the 'weather' array
        for idx, j in enumerate(city_weather["weather"]):
            # Add a comma if there is more than one description value.
            separator = '' if idx == 0 else ', '
            city_weather_desc = city_weather_desc + separator + j['description'] 
            #print (city_weather_desc)
        # amount of rainfall over the last hour (1 hr) - WHERE AVAILABLE!
        # amount of snowfall over the last hour (1 hr) - WHERE AVAILABLE!
        # Create endpoint URL for each city's lat's/long's.
        weather_url = weather_base_url + "&lat=" + str(city_lat) + "&lon=" + str(city_lng)
        #print(weather_url)
        # Make the Weather API call, parse the JSON response (assuming you get the expected response) and retrieve data.
        weather_forecast = requests.get(weather_url).json()
        #print(weather_forecast)
        try:
            current_rain = weather_forecast['current']['rain']['1h']
        except:
            current_rain = 0
        #print (current_rain)
        try:
            current_snow = weather_forecast['current']['snow']['1h']
        except:
            current_snow = 0
        #print (current_snow)
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city & weather information into city_weather_data list.

        # -- Latitude and longitude
        # -- Maximum temperature
        # -- Percent humidity
        # -- Percent cloudiness
        # -- Wind speed
        # -- Weather description (e.g., clouds, fog, light rain, clear sky)
        # -- The amount of rainfall over the last hour (1 hr).
        # -- The amount of snowfall over the last hour (1 hr)
        # Also include city & date for reference.
        city_weather_data.append({
            "City": city.title(),
            "Lat": city_lat,
            "Lng": city_lng,
            "Max Temp": city_max_temp,
            "Humidity": city_humidity,
            "Cloudiness": city_clouds,
            "Wind Speed": city_wind,
            "Country": city_country,
            "Weather" : city_weather_desc,
            "Rain last hour": current_rain,
            "Snow last hour" : current_snow,
            "Date": city_date
        })

    # If an error is experienced, skip the city.
    except KeyError as e:
        print(f"Exception Type {type(e)}, {e}")
        print(f"City {city} not found. Skipping...")
        pass
    except Exception as e:
        print(f"Exception Type {type(e)}, {e}")
        print(f"City {city} OTHER EXCEPTION. RECHECK!!")
        pass
    
    
# Indicate that Data Loading is complete.
print("-----------------------------")
print(f"Processed {total_count} records, {set_count} number of Sets")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | albany
Processing Record 2 of Set 1 | ostrovnoy
Processing Record 3 of Set 1 | beba
Processing Record 4 of Set 1 | marcona
Exception Type <class 'KeyError'>, 'coord'
City marcona not found. Skipping...
Processing Record 5 of Set 1 | qaanaaq
Processing Record 6 of Set 1 | hilo
Processing Record 7 of Set 1 | rikitea
Processing Record 8 of Set 1 | alice springs
Processing Record 9 of Set 1 | atuona
Processing Record 10 of Set 1 | carnarvon
Processing Record 11 of Set 1 | doha
Processing Record 12 of Set 1 | punta arenas
Processing Record 13 of Set 1 | phichit
Processing Record 14 of Set 1 | lompoc
Processing Record 15 of Set 1 | grand river south east
Exception Type <class 'KeyError'>, 'coord'
City grand river south east not found. Skipping...
Processing Record 16 of Set 1 | paikuse
Processing Record 17 of Set 1 | ravar
Processing Record 18 of Set 1 | muscat
Processing Record 19 of Set 1 | bridlingt

Processing Record 18 of Set 4 | sentyabrskiy
Exception Type <class 'KeyError'>, 'coord'
City sentyabrskiy not found. Skipping...
Processing Record 19 of Set 4 | coruripe
Processing Record 20 of Set 4 | kamenskoye
Exception Type <class 'KeyError'>, 'coord'
City kamenskoye not found. Skipping...
Processing Record 21 of Set 4 | kindu
Processing Record 22 of Set 4 | acatlan
Processing Record 23 of Set 4 | kazalinsk
Exception Type <class 'KeyError'>, 'coord'
City kazalinsk not found. Skipping...
Processing Record 24 of Set 4 | ankazoabo
Processing Record 25 of Set 4 | husavik
Processing Record 26 of Set 4 | dongsheng
Processing Record 27 of Set 4 | hoa binh
Processing Record 28 of Set 4 | tautira
Processing Record 29 of Set 4 | tabialan
Exception Type <class 'KeyError'>, 'coord'
City tabialan not found. Skipping...
Processing Record 30 of Set 4 | itarema
Processing Record 31 of Set 4 | florence
Processing Record 32 of Set 4 | bethel
Processing Record 33 of Set 4 | sur
Processing Record 34 o

Processing Record 31 of Set 7 | katobu
Processing Record 32 of Set 7 | rawson
Processing Record 33 of Set 7 | rafsanjan
Processing Record 34 of Set 7 | grindavik
Processing Record 35 of Set 7 | pevek
Processing Record 36 of Set 7 | kudahuvadhoo
Processing Record 37 of Set 7 | senanga
Processing Record 38 of Set 7 | westpunt
Exception Type <class 'KeyError'>, 'coord'
City westpunt not found. Skipping...
Processing Record 39 of Set 7 | saldanha
Processing Record 40 of Set 7 | cache creek
Processing Record 41 of Set 7 | juegang
Processing Record 42 of Set 7 | yerbogachen
Processing Record 43 of Set 7 | zaysan
Processing Record 44 of Set 7 | grand gaube
Processing Record 45 of Set 7 | kochevo
Processing Record 46 of Set 7 | namibe
Processing Record 47 of Set 7 | genhe
Processing Record 48 of Set 7 | hambantota
Processing Record 49 of Set 7 | archidona
Processing Record 50 of Set 7 | kupino
Processing Record 1 of Set 8 | batagay-alyta
Processing Record 2 of Set 8 | llangefni
Processing Reco

Processing Record 4 of Set 11 | bekhtery
Processing Record 5 of Set 11 | ust-kamchatsk
Exception Type <class 'KeyError'>, 'coord'
City ust-kamchatsk not found. Skipping...
Processing Record 6 of Set 11 | tilichiki
Processing Record 7 of Set 11 | richards bay
Processing Record 8 of Set 11 | bardiyah
Processing Record 9 of Set 11 | anadyr
Processing Record 10 of Set 11 | sabang
Processing Record 11 of Set 11 | clyde river
Processing Record 12 of Set 11 | yangjiang
Processing Record 13 of Set 11 | koslan
Processing Record 14 of Set 11 | gazanjyk
Processing Record 15 of Set 11 | auki
Processing Record 16 of Set 11 | ternate
Processing Record 17 of Set 11 | sug-aksy
Exception Type <class 'KeyError'>, 'coord'
City sug-aksy not found. Skipping...
Processing Record 18 of Set 11 | noyabrsk
Processing Record 19 of Set 11 | raudeberg
Processing Record 20 of Set 11 | popondetta
Processing Record 21 of Set 11 | sechura
Processing Record 22 of Set 11 | lishu
Processing Record 23 of Set 11 | catuday


In [26]:
# 6. Add the data to a new DataFrame.
city_weather_data_df=pd.DataFrame(city_weather_data)

In [28]:
# From "Module 6-Detailed-Rubric-Python-API_online_rubric-1.pdf"
## ✓ Use Pandas to correctly answer this question: 
### How many cities have recorded rainfall or snow?
print ((city_weather_data_df.loc[city_weather_data_df['Rain last hour'] > 0].count()))
print ((city_weather_data_df.loc[city_weather_data_df['Snow last hour'] > 0].count()))


City              70
Lat               70
Lng               70
Max Temp          70
Humidity          70
Cloudiness        70
Wind Speed        70
Country           70
Weather           70
Rain last hour    70
Snow last hour    70
Date              70
dtype: int64
City              0
Lat               0
Lng               0
Max Temp          0
Humidity          0
Cloudiness        0
Wind Speed        0
Country           0
Weather           0
Rain last hour    0
Snow last hour    0
Date              0
dtype: int64


In [13]:
# 7. Save the new DataFrame as a CSV file to be used for Part 2.
# Create the output file (CSV).
output_data_file = "weather_data/cities_weather.csv"
# Export the city_weather_data into a CSV.
city_weather_data_df.to_csv(output_data_file, index_label="City_ID")

# 8. Upload the CSV file as part of your submission as WeatherPy_database.csv.
# - DONE, see https://github.com/damiencorr/World_Weather_Analysis/tree/master/weather_data